In [6]:
import transformers

model = transformers.AutoModelForCausalLM.from_pretrained(
        "gpt2"
    )


/home/ubuntu/miniconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/home/ubuntu/miniconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
model.transformer.h

ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D()
      (c_proj): Conv1D()
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [4]:
import torch
import torch.nn.functional as F
from torch import nn
import time
from tqdm import tqdm

class FeedForward(nn.Module):

    def __init__(self, in_features, mid_features, dropout=0.1):
        super(FeedForward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, mid_features),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mid_features, in_features),
        )
    
    def forward(self, x):
        return self.net(x)

class Network(torch.nn.Module):

    def __init__(self, dim):
        super(Network, self).__init__()
        self.model = nn.Sequential(
            FeedForward(dim, dim*4),
            FeedForward(dim, dim*4),
            FeedForward(dim, dim*4),
            FeedForward(dim, dim*4),
        )
    
    def forward(self, x):
        return self.model(x)

    
net = Network(1024).cuda().to(torch.float16).requires_grad_(False)
net = torch.compile(net)

x = torch.randn(64*15, 1024).cuda().to(torch.float16)

times = []
for i in range(10):
    y = net(x)


for i in tqdm(range(5000)):
    start = time.time()
    y = net(x)
    times.append(time.time() - start)

print(sum(times)/len(times))


100%|██████████| 5000/5000 [00:03<00:00, 1280.97it/s]

0.0007665396213531494
